# Transfer Learning CIFAR10

* Train a simple convnet on the CIFAR dataset the first 5 output classes [0..4].
* Freeze convolutional layers and fine-tune dense layers for the last 5 ouput classes [5..9].


### 1. Import CIFAR10 data and create 2 datasets with one dataset having classes from 0 to 4 and other having classes from 5 to 9 

In [2]:
import tensorflow as tf
tf.reset_default_graph()
tf.set_random_seed(42)

(X_train,Y_train),(X_test,Y_test)  = tf.keras.datasets.cifar10.load_data()


170500096/170498071 [==============================] - 6s 0us/step


In [3]:
X_train.shape

(50000, 32, 32, 3)

In [0]:
Y_train = Y_train.flatten()
Y_test = Y_test.flatten()

In [0]:
X_train_cf5 = X_train[Y_train < 5]
Y_train_cf5 = Y_train[Y_train < 5]
X_test_cf5 = X_test[Y_test < 5]
Y_test_cf5 = Y_test[Y_test < 5]

X_train_ct5 = X_train[Y_train >= 5]
Y_train_ct5 = Y_train[Y_train >= 5] - 5  
X_test_ct5 = X_test[Y_test >= 5]         
Y_test_ct5 = Y_test[Y_test >= 5] - 5

### 2. Use One-hot encoding to divide y_train and y_test into required no of output classes

In [0]:
Y_train_cf5 = tf.keras.utils.to_categorical(Y_train_cf5,num_classes=5)
Y_test_cf5 = tf.keras.utils.to_categorical(Y_test_cf5,num_classes=5)
Y_train_ct5 = tf.keras.utils.to_categorical(Y_train_ct5,num_classes=5)
Y_test_ct5 = tf.keras.utils.to_categorical(Y_test_ct5,num_classes=5)

### 3. Build a sequential neural network model which can classify the classes 0 to 4 of CIFAR10 dataset with at least 80% accuracy on test data

In [0]:
X_train_cf5 = X_train_cf5.astype('float32') / 255
X_test_cf5 = X_test_cf5.astype('float32') / 255

In [0]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Reshape
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization

In [19]:
# Define model
    model = Sequential()

    # 1st Conv Layer
    model.add(Convolution2D(64, 3, 3, input_shape=(32, 32, 3)))
    model.add(Activation('relu'))

    # 2nd Conv Layer
    model.add(Convolution2D(128, 3, 3))
    model.add(Activation('relu'))
    
    # Max Pooling
    model.add(MaxPooling2D(pool_size=(2,2)))
    
    # Dropout layer
    model.add(Dropout(0.25))
    

    # Fully Connected Layer
    model.add(Flatten())
    model.add(Dense(128))
    model.add(Activation('relu'))
    model.add(BatchNormalization())

    # Prediction Layer
    model.add(Dense(5))
    model.add(Activation('softmax'))
    
    # Loss and Optimizer
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    # Train the model
    model.fit(X_train_cf5, Y_train_cf5, batch_size=32, nb_epoch=10, 
              validation_data=(X_test_cf5, Y_test_cf5))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), input_shape=(32, 32, 3...)`
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3))`
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:33: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 25000 samples, validate on 5000 samples
Epoch 1/10
25000/25000 [==============================] - 17s 691us/step - loss: 0.9031 - acc: 0.6434 - val_loss: 0.7881 - val_acc: 0.6884
Epoch 2/10
25000/25000 [==============================] - 16s 645us/step - loss: 0.6976 - acc: 0.7374 - val_loss: 0.6889 - val_acc: 0.7406
Epoch 3/10
25000/25000 [==============================] - 16s 647us/step - loss: 0.6118 - acc: 0.7686 - val_loss: 0.7673 - val_acc: 0.6850
Epoch 4/10
25000/25000 [==============================] - 16s 645us/step - loss: 0.5444 - acc: 0.7924 - val_loss: 0.6277 - val_acc: 0.7718
Epoch 5/10
25000/25000 [==============================] - 16s 647us/step - loss: 0.4748 - acc: 0.8192 - val_loss: 0.6453 - val_acc: 0.7696
Epoch 6/10
25000/25000 [==============================] - 16s 646us/step - loss: 0.3935 - acc: 0.8536 - val_loss: 0.6338 - val_acc: 0.7780
Epoch 7/10
25000/25000 [==============================] - 16s 646us/step - loss: 0.3267 - acc: 0.8774 - val_loss: 0.6

In [20]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_13 (Conv2D)           (None, 30, 30, 64)        1792      
_________________________________________________________________
activation_25 (Activation)   (None, 30, 30, 64)        0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 28, 28, 128)       73856     
_________________________________________________________________
activation_26 (Activation)   (None, 28, 28, 128)       0         
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 14, 14, 128)       0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 14, 14, 128)       0         
_________________________________________________________________
flatten_7 (Flatten)          (None, 25088)             0         
__________

In [26]:
from keras.preprocessing.image import ImageDataGenerator

EPOCHS = 15
BS = 32

# construct the training image generator for data augmentation
aug = ImageDataGenerator(rotation_range=20, zoom_range=0.15,
	width_shift_range=0.2, height_shift_range=0.2, shear_range=0.15,
	horizontal_flip=True, fill_mode="nearest")
 
# train the network
model.fit_generator(aug.flow(X_train_cf5, Y_train_cf5, batch_size=BS),
	validation_data=(X_test_cf5, Y_test_cf5), steps_per_epoch=len(X_train_cf5) // BS,
	epochs=EPOCHS)

Epoch 1/15
  6/781 [..............................] - ETA: 22s - loss: 0.7015 - acc: 0.7552

/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


781/781 [==============================] - 25s 32ms/step - loss: 0.6696 - acc: 0.7471 - val_loss: 0.6344 - val_acc: 0.7644
Epoch 2/15
781/781 [==============================] - 26s 33ms/step - loss: 0.6625 - acc: 0.7474 - val_loss: 0.5684 - val_acc: 0.7862
Epoch 3/15
781/781 [==============================] - 26s 34ms/step - loss: 0.6538 - acc: 0.7509 - val_loss: 0.5743 - val_acc: 0.7880
Epoch 4/15
781/781 [==============================] - 26s 33ms/step - loss: 0.6466 - acc: 0.7536 - val_loss: 0.5652 - val_acc: 0.7950
Epoch 5/15
781/781 [==============================] - 26s 33ms/step - loss: 0.6318 - acc: 0.7620 - val_loss: 0.5312 - val_acc: 0.8048
Epoch 6/15
781/781 [==============================] - 26s 33ms/step - loss: 0.6284 - acc: 0.7625 - val_loss: 0.5439 - val_acc: 0.7986
Epoch 7/15
781/781 [==============================] - 26s 33ms/step - loss: 0.6227 - acc: 0.7662 - val_loss: 0.5357 - val_acc: 0.8054
Epoch 8/15
781/781 [==============================] - 26s 33ms/step - los

### 4. In the model which was built above (for classification of classes 0-4 in CIFAR10), make only the dense layers to be trainable and conv layers to be non-trainable

In [27]:
for layers in model.layers:
    print(layers.name)
    if('dense' not in layers.name):
        layers.trainable = False
        print(layers.name + 'is not trainable\n')
    if('dense' in layers.name):
        print(layers.name + ' is trainable\n')

conv2d_13
conv2d_13is not trainable

activation_25
activation_25is not trainable

conv2d_14
conv2d_14is not trainable

activation_26
activation_26is not trainable

max_pooling2d_7
max_pooling2d_7is not trainable

dropout_7
dropout_7is not trainable

flatten_7
flatten_7is not trainable

dense_13
dense_13 is trainable

activation_27
activation_27is not trainable

batch_normalization_4
batch_normalization_4is not trainable

dense_14
dense_14 is trainable

activation_28
activation_28is not trainable



### 5. Utilize the the model trained on CIFAR 10 (classes 0 to 4) to classify the classes 5 to 9 of CIFAR 10  (Use Transfer Learning) <br>
Achieve an accuracy of more than 85% on test data

In [0]:
X_train_ct5 = X_train_ct5.astype('float32') / 255
X_test_ct5 = X_test_ct5.astype('float32') / 255

In [0]:
EPOCHS = 15
BS = 32

In [0]:
aug_1 = ImageDataGenerator(rotation_range=20, zoom_range=0.25,
	width_shift_range=0.1, height_shift_range=0.1, shear_range=0.15,
	horizontal_flip=True, fill_mode="nearest")

In [37]:
# train the network
model.fit_generator(aug_1.flow(X_train_ct5, Y_train_ct5, batch_size=BS),
	validation_data=(X_test_ct5, Y_test_ct5), steps_per_epoch=len(X_train_ct5) // BS,
	epochs=EPOCHS)

Epoch 1/10
  7/781 [..............................] - ETA: 21s - loss: 1.1607 - acc: 0.5312

/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


781/781 [==============================] - 25s 32ms/step - loss: 1.1537 - acc: 0.5314 - val_loss: 1.0765 - val_acc: 0.5718
Epoch 2/10
781/781 [==============================] - 26s 33ms/step - loss: 1.1555 - acc: 0.5318 - val_loss: 1.1451 - val_acc: 0.5284
Epoch 3/10
781/781 [==============================] - 26s 33ms/step - loss: 1.1520 - acc: 0.5324 - val_loss: 1.0638 - val_acc: 0.5752
Epoch 4/10
781/781 [==============================] - 26s 33ms/step - loss: 1.1581 - acc: 0.5246 - val_loss: 1.0547 - val_acc: 0.5772
Epoch 5/10
781/781 [==============================] - 26s 33ms/step - loss: 1.1541 - acc: 0.5356 - val_loss: 1.0855 - val_acc: 0.5648
Epoch 6/10
781/781 [==============================] - 26s 33ms/step - loss: 1.1507 - acc: 0.5337 - val_loss: 1.1233 - val_acc: 0.5418
Epoch 7/10
781/781 [==============================] - 26s 33ms/step - loss: 1.1523 - acc: 0.5329 - val_loss: 1.0398 - val_acc: 0.5824
Epoch 8/10
781/781 [==============================] - 26s 33ms/step - los

In [38]:
score = model.evaluate(X_test_ct5, Y_test_ct5, batch_size=128, verbose=0)
print(score)

[1.0933497990608216, 0.563]


## Sentiment analysis <br> 

The objective of the second problem is to perform Sentiment analysis from the tweets data collected from the users targeted at various mobile devices.
Based on the tweet posted by a user (text), we will classify if the sentiment of the user targeted at a particular mobile device is positive or not.

### 6. Read the dataset (tweets.csv) and drop the NA's while reading the dataset

In [62]:
from google.colab import files
uploaded = files.upload()

Saving tweets-1.csv to tweets-1 (2).csv


In [0]:
import pandas as pd
import io

import io
data = pd.read_csv(io.BytesIO(uploaded['tweets-1.csv']),encoding = "ISO-8859-1").dropna()  

In [66]:
data.shape

(3291, 3)

In [67]:
data.head()

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion


### Consider only rows having Positive emotion and Negative emotion and remove other rows from the dataframe.

In [0]:
data = data[(data['is_there_an_emotion_directed_at_a_brand_or_product'] == 'Positive emotion') | (data['is_there_an_emotion_directed_at_a_brand_or_product'] == 'Negative emotion')]

In [69]:
data.shape

(3191, 3)

### 7. Represent text as numerical data using `CountVectorizer` and get the document term frequency matrix

#### Use `vect` as the variable name for initialising CountVectorizer.

In [0]:
X = data['tweet_text']

In [71]:
X.shape

(3191,)

In [0]:
Y = data['is_there_an_emotion_directed_at_a_brand_or_product']

In [0]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [0]:
vect = CountVectorizer()
tf = vect.fit_transform(X)

### 8. Find number of different words in vocabulary

In [75]:
tf.shape

(3191, 5648)

#### Tip: To see all available functions for an Object use dir

In [77]:
dir(tf)

['__abs__',
 '__add__',
 '__array_priority__',
 '__bool__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__div__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__idiv__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__isub__',
 '__iter__',
 '__itruediv__',
 '__le__',
 '__len__',
 '__lt__',
 '__matmul__',
 '__module__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__nonzero__',
 '__pow__',
 '__radd__',
 '__rdiv__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rmatmul__',
 '__rmul__',
 '__rsub__',
 '__rtruediv__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__sub__',
 '__subclasshook__',
 '__truediv__',
 '__weakref__',
 '_add_dense',
 '_add_sparse',
 '_arg_min_or_max',
 '_arg_min_or_max_axis',
 '_asindices',
 '_binopt',
 '_cs_matrix__get_has_canonical_format',
 '_cs_matrix__get_sorted',
 '_cs_matrix__set_has_canonical_format',
 '_cs_matrix__set_sorted

### Find out how many Positive and Negative emotions are there.

Hint: Use value_counts on that column

In [78]:
pd.value_counts(data['is_there_an_emotion_directed_at_a_brand_or_product'])

Positive emotion    2672
Negative emotion     519
Name: is_there_an_emotion_directed_at_a_brand_or_product, dtype: int64

###  Change the labels for Positive and Negative emotions as 1 and 0 respectively and store in a different column in the same dataframe named 'label'

Hint: use map on that column and give labels

In [0]:
data['label'] = data.is_there_an_emotion_directed_at_a_brand_or_product.map({'Positive emotion':1, 'Negative emotion':0})

### 9. Define the feature set (independent variable or X) to be `text` column and `labels` as target (or dependent variable)  and divide into train and test datasets

In [0]:
x = data.tweet_text
y = data['label']

In [0]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(x,y, random_state= 24)

## 10. **Predicting the sentiment:**


### Use Naive Bayes and Logistic Regression and their accuracy scores for predicting the sentiment of the given text

In [0]:
vect = CountVectorizer(ngram_range=(1, 1))
X_train_dtm = vect.fit_transform(xtrain)
X_test_dtm = vect.transform(xtest)

In [0]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()

In [84]:
nb.fit(X_train_dtm,ytrain)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [0]:
y_pred = nb.predict(X_test_dtm)

In [86]:
from sklearn import metrics
metrics.accuracy_score(ytest, y_pred)

0.849624060150376

In [0]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()

In [88]:
lr.fit(X_train_dtm,ytrain)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [0]:
y_pred_lr = lr.predict(X_test_dtm)

In [90]:
metrics.accuracy_score(ytest, y_pred_lr)

0.8571428571428571

## 11. Create a function called `tokenize_predict` which can take count vectorizer object as input and prints the accuracy for x (text) and y (labels)

In [0]:
def tokenize_test(vect):
    X_train_dtm = vect.fit_transform(xtrain)
    print('Features: ', X_train_dtm.shape[1])
    X_test_dtm = vect.transform(xtest)
    nb = MultinomialNB()
    nb.fit(X_train_dtm, ytrain)
    y_pred_class = nb.predict(X_test_dtm)
    print('Accuracy: ', metrics.accuracy_score(ytest, y_pred_class))

### Create a count vectorizer function which includes n_grams = 1,2  and pass it to tokenize_predict function to print the accuracy score

In [96]:
# include 1-grams and 2-grams
vect = CountVectorizer(ngram_range=(1, 2))
tokenize_test(vect)

Features:  24420
Accuracy:  0.8583959899749374


### 12. Create a count vectorizer function with stopwords = 'english'  and pass it to tokenize_predict function to print the accuracy score

In [97]:
vect1 = CountVectorizer(stop_words='english')
tokenize_test(vect1)

Features:  4609
Accuracy:  0.849624060150376


### 13. Create a count vectorizer function with stopwords = 'english' and max_features =300  and pass it to tokenize_predict function to print the accuracy score

In [98]:
vect2 = CountVectorizer(stop_words='english',max_features =300)
tokenize_test(vect2)

Features:  300
Accuracy:  0.8157894736842105


### 14. Create a count vectorizer function with n_grams = 1,2  and max_features = 15000  and pass it to tokenize_predict function to print the accuracy score

In [99]:
vect3 = CountVectorizer(ngram_range=(1, 2),max_features =15000)
tokenize_test(vect3)

Features:  15000
Accuracy:  0.8521303258145363


### 15. Create a count vectorizer function with n_grams = 1,2  and include terms that appear at least 2 times (min_df = 2)  and pass it to tokenize_predict function to print the accuracy score

In [100]:
vect4 = CountVectorizer(ngram_range=(1, 2),min_df = 2)
tokenize_test(vect4)

Features:  7655
Accuracy:  0.8483709273182958
